# Using Ragas to Evaluate a RAG Application built with LangChain and LangGraph

In the following notebook, we'll be looking at how [Ragas](https://github.com/explodinggradients/ragas) can be helpful in a number of ways when looking to evaluate your RAG applications!

While this example is rooted in LangChain/LangGraph - Ragas is framework agnostic (you don't even need to be using a framework!).

- 🤝 Breakout Room #1
  1. Task 1: Installing Required Libraries
  2. Task 2: Set Environment Variables
  3. Task 3: Synthetic Dataset Generation for Evaluation using Ragas (Optional)

- 🤝 Breakout Room #2
  1. Task 4: Evaluating our Pipeline with Ragas
  2. Task 5: Testing OpenAI's Claim
  3. Task 6: Selecting an Advanced Retriever and Evaluating

But first! Let's set some dependencies!

## Dependencies and API Keys:

In [ ]:
!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

We'll also need to provide our API keys.

First, OpenAI's for our LLM/embedding model combination!

In [ ]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key!")

Please enter your Ragas API key!··········


OPTIONALLY:

We can also provide a Ragas API key - which you can sign-up for [here](https://app.ragas.io/).

In [ ]:
os.environ["RAGAS_APP_TOKEN"] = getpass("Please enter your Ragas API key!")

Please enter your OpenAI API key!··········


## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [ ]:
!mkdir data

In [ ]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31330    0 31330    0     0   122k      0 --:--:-- --:--:-- --:--:--  122k


In [ ]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 69985    0 69985    0     0   259k      0 --:--:-- --:--:-- --:--:--  260k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [ ]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [ ]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Wuz the best LLM trained in China for less tha...,[Prompt driven app generation is a commodity a...,The context mentions a question about whether ...,single_hop_specifc_query_synthesizer
1,How has OpenAI contributed to the advancements...,"[gets you OpenAI’s most expensive model, o1. G...",OpenAI has significantly contributed to the ad...,single_hop_specifc_query_synthesizer
2,what was the problem with openai's websocket api?,[feed with the model and talk about what you c...,OpenAI started with a WebSocket API that was q...,single_hop_specifc_query_synthesizer
3,What Anthropic do?,[dependent on AGI itself. A model that’s robus...,Anthropic is involved in developing robust mod...,single_hop_specifc_query_synthesizer
4,"How do the ethics of AI, particularly concerni...",[<1-hop>\n\nCode may be the best application T...,"The ethics of AI, especially regarding trainin...",multi_hop_abstract_query_synthesizer
5,What are the ethical concerns related to the t...,[<1-hop>\n\nCode may be the best application T...,The ethical concerns related to the training d...,multi_hop_abstract_query_synthesizer
6,What are the ethical considerations surroundin...,[<1-hop>\n\nCode may be the best application T...,The ethical considerations surrounding the tra...,multi_hop_abstract_query_synthesizer
7,How do the challenges of understanding and con...,[<1-hop>\n\nCode may be the best application T...,The challenges of understanding and controllin...,multi_hop_abstract_query_synthesizer
8,How does the development of Claude 3 and other...,[<1-hop>\n\ngets you OpenAI’s most expensive m...,The development of Claude 3 and other multi-mo...,multi_hop_specific_query_synthesizer
9,What are the cost and efficiency benefits of u...,[<1-hop>\n\nThose of us who understand this st...,"In 2024, the cost and efficiency benefits of u...",multi_hop_specific_query_synthesizer


#### OPTIONAL:

If you've provided your Ragas API key - you can use this web interface to look at the created data!

In [ ]:
dataset.upload()

Testset uploaded! View at https://app.ragas.io/dashboard/alignment/testset/e891eb9e-ebcb-442a-8dea-c57785e81775


'https://app.ragas.io/dashboard/alignment/testset/e891eb9e-ebcb-442a-8dea-c57785e81775'

## LangChain RAG

Now we'll construct our LangChain RAG, which we will be evaluating using the above created test data!

### R - Retrieval

Let's start with building our retrieval pipeline, which will involve loading the same data we used to create our synthetic test set above.

> NOTE: We need to use the same data - as our test set is specifically designed for this data.

In [ ]:
path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

Now that we have our data loaded, let's split it into chunks!

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

74

Next up, we'll need to provide an embedding model that we can use to construct our vector store.

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

Now we can build our in memory QDrant vector store.

In [ ]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="ai_across_years",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="ai_across_years",
    embedding=embeddings,
)

We can now add our documents to our vector store.

In [ ]:
_ = vector_store.add_documents(documents=split_documents)

['de9bb2769fe5498692a34bb412cb6b77',
 '3d2e73bcabac43e58c2ccaf7fba3d941',
 '5dd61910206f4861b18e89ee033187c4',
 '04e697eb9865488e92aafd5c7ba3148b',
 'dee12306ea3d49e6b4bd543048ae19fb',
 'aec904188ad943b8b3675f69587f67a8',
 'efd250477495477b97e38e8f392a502f',
 '389a63a73e5b42fd8f5aa385afcc01df',
 '4421e49fdb22404fa38ab5826a5e187d',
 '6809544d0a514a1e939f4a8daafc44e0',
 'c2251e1bdba2489e8ea128b5b98cc66a',
 '18e7eab42f4648818cc01f808c237d85',
 'c3711a14ce284b1f87678fe854cea140',
 'f1c854dc205e43ff8ddca6dc51d95966',
 '7265552dce8242d8862ba620d45cd492',
 '77240ca56dc149ccad0166521f3ebe51',
 'ae55ae2f25d642dab305908f8df56386',
 'e0d53aa2f9ef4e5a9819c296ee018fa1',
 '4f4538872e7942928acafb24fbacdd63',
 '0dd71b43033149b3ae7e9ded266ff20e',
 'c247fe92546a46a5b4651792ceec0818',
 '2065a02ee104451b87d7f58c4143c690',
 'f1771dea255244449df358d5d774274c',
 '89399bb5f98244a6b266864750514efa',
 '2e781fe8437b4a7f876473e97c5b20d8',
 '94477274e38e4f819ff3caf6c0101f22',
 'e2a1e2a50cf144789ae4ca8b20f488d1',
 

Let's define our retriever.

In [ ]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

Now we can produce a node for retrieval!

In [ ]:
def retrieve(state):
  retrieved_docs = retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

### Augmented

Let's create a simple RAG prompt!

In [ ]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

### Generation

We'll also need an LLM to generate responses - we'll use `gpt-4o-mini` to avoid using the same model as our judge model.

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Then we can create a `generate` node!

In [ ]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

### Building RAG Graph with LangGraph

Let's create some state for our LangGraph RAG graph!

In [ ]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

Now we can build our simple graph!

> NOTE: We're using `add_sequence` since we will always move from retrieval to generation. This is essentially building a chain in LangGraph.

In [ ]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

Let's do a test to make sure it's doing what we'd expect.

In [ ]:
response = graph.invoke({"question" : "How are LLM agents useful?"})

In [ ]:
response["response"]

'LLM agents are useful primarily because they can assist in various tasks, particularly in code generation, where they demonstrate significant capability. They can generate code and even execute it to verify correctness, which mitigates the issue of hallucination (producing incorrect information) that is more problematic in natural language processing. \n\nAdditionally, LLMs are relatively easy to build compared to expectations, requiring only a few hundred lines of code, provided one has access to sufficient and quality training data. This accessibility means that while training an LLM may not be feasible for hobbyists, it is no longer limited to only those with vast resources.\n\nMoreover, LLMs can now be run on personal devices, making them more widely usable. This democratization of access allows more people to leverage their potential.\n\nDespite their usefulness, LLMs warrant criticism regarding ethical considerations, reliability, and their environmental impact, suggesting that 

## Evaluating the App with Ragas

Now we can finally do our evaluation!

We'll start by running the queries we generated usign SDG above through our application to get context and responses.

In [ ]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [ ]:
dataset.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,synthesizer_name
0,Wuz the best LLM trained in China for less tha...,[Was the best currently available LLM trained ...,[Prompt driven app generation is a commodity a...,The best currently available LLM trained in Ch...,The context mentions a question about whether ...,single_hop_specifc_query_synthesizer
1,How has OpenAI contributed to the advancements...,"[In 2024, almost every significant model vendo...","[gets you OpenAI’s most expensive model, o1. G...",OpenAI has significantly contributed to the ad...,OpenAI has significantly contributed to the ad...,single_hop_specifc_query_synthesizer
2,what was the problem with openai's websocket api?,[Did you know ChatGPT has two entirely differe...,[feed with the model and talk about what you c...,The problem with OpenAI's WebSocket API was th...,OpenAI started with a WebSocket API that was q...,single_hop_specifc_query_synthesizer
3,What Anthropic do?,[Getting back to models that beat GPT-4: Anthr...,[dependent on AGI itself. A model that’s robus...,Anthropic is involved in developing advanced A...,Anthropic is involved in developing robust mod...,single_hop_specifc_query_synthesizer
4,"How do the ethics of AI, particularly concerni...","[Since then, almost every major LLM (and most ...",[<1-hop>\n\nCode may be the best application T...,"The ethics of AI, particularly concerning trai...","The ethics of AI, especially regarding trainin...",multi_hop_abstract_query_synthesizer
5,What are the ethical concerns related to the t...,"[Since then, almost every major LLM (and most ...",[<1-hop>\n\nCode may be the best application T...,The ethical concerns related to the training d...,The ethical concerns related to the training d...,multi_hop_abstract_query_synthesizer
6,What are the ethical considerations surroundin...,"[Since then, almost every major LLM (and most ...",[<1-hop>\n\nCode may be the best application T...,The ethical considerations surrounding the tra...,The ethical considerations surrounding the tra...,multi_hop_abstract_query_synthesizer
7,How do the challenges of understanding and con...,[Another common technique is to use larger mod...,[<1-hop>\n\nCode may be the best application T...,The challenges of understanding and controllin...,The challenges of understanding and controllin...,multi_hop_abstract_query_synthesizer
8,How does the development of Claude 3 and other...,"[In 2024, almost every significant model vendo...",[<1-hop>\n\ngets you OpenAI’s most expensive m...,The development of Claude 3 and other multi-mo...,The development of Claude 3 and other multi-mo...,multi_hop_specific_query_synthesizer
9,What are the cost and efficiency benefits of u...,[This increase in efficiency and reduction in ...,[<1-hop>\n\nThose of us who understand this st...,The cost and efficiency benefits of using Clau...,"In 2024, the cost and efficiency benefits of u...",multi_hop_specific_query_synthesizer


Then we can convert that table into a `EvaluationDataset` which will make the process of evaluation smoother.

In [ ]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

We'll need to select a judge model - in this case we're using the same model that was used to generate our Synthetic Data.

In [ ]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

Next up - we simply evaluate on our desired metrics!

In [ ]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[11]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[29]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[35]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[41]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[47]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[53]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[59]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[65]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[71]: TimeoutError()


{'context_recall': 0.6062, 'faithfulness': 0.6913, 'factual_correctness': 0.4183, 'answer_relevancy': 0.9587, 'context_entity_recall': 0.4072, 'noise_sensitivity_relevant': 0.3611}

## Making Adjustments and Re-Evaluating

Now that we've got our baseline - let's make a change and see how the model improves or doesn't improve!

> NOTE: This will be using Cohere's Rerank model - please be sure to sign-up for an API key!

In [ ]:
os.environ["COHERE_API_KEY"] = getpass("Please enter your Cohere API key!")

Please enter your Cohere API key!··········


In [ ]:
!pip install -qU cohere langchain_cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 11.5 MB/s eta 0:00:00


In [ ]:
retriever = vector_store.as_retriever(search_kwargs={"k": 20})

In [ ]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

def retrieve_adjusted(state):
  compressor = CohereRerank(model="rerank-english-v3.0")
  compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever, search_kwargs={"k": 5}
  )
  retrieved_docs = compression_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

In [ ]:
class State(TypedDict):
  question: str
  context: List[Document]
  response: str

graph_builder = StateGraph(State).add_sequence([retrieve_adjusted, generate])
graph_builder.add_edge(START, "retrieve_adjusted")
graph = graph_builder.compile()

In [ ]:
response = graph.invoke({"question" : "How are LLM agents useful?"})
response["response"]

'LLM agents are seen as potentially useful for automating tasks and acting on behalf of users, particularly in areas like writing code. Their ability to generate code is highlighted as one of their strongest capabilities due to the relatively simpler grammar rules of programming languages compared to natural languages. However, there is significant skepticism regarding their overall utility, especially because LLMs struggle with distinguishing truth from fiction, which can lead to unreliable outputs. Critics point out the challenges associated with gullibility and the potential negative consequences of relying on such systems. Despite the excitement around AI agents, there are few examples of them in practical use, indicating that their effectiveness in real-world applications remains uncertain.'

In [ ]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [ ]:
result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[11]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[29]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[35]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[41]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[47]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[53]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[59]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[65]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[71]: TimeoutError()


{'context_recall': 0.5645, 'faithfulness': 0.7479, 'factual_correctness': 0.4142, 'answer_relevancy': 0.9574, 'context_entity_recall': 0.3944, 'noise_sensitivity_relevant': 0.5093}